In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
task_names = [
    "COLA", 
#     "SST2", 
#     "MNLI",
#     "RTE",
#     "WNLI",
#     "QQP",
#     "MRPC",
#     "STSB",
#     "QNLI",
#     "SPACY_POS",
#     "SPACY_NER",
#     "THIRD",
#     "BLEU",
]

FILENAME = "COLA_tasks_and_payloads"

In [3]:
SEED = 1

## Baseline: Train COLA Task

In [4]:
%%time
from metal.mmtl.glue.glue_tasks import create_tasks_and_payloads

# Create tasks and payloads
tasks, payloads = create_tasks_and_payloads(
    task_names,
    dl_kwargs={"batch_size": 16}
)

Using random seed: 420412
Loading COLA Dataset



CPU times: user 13.3 s, sys: 6.24 s, total: 19.5 s
Wall time: 15.4 s


In [ ]:
tasks, payloads

([ClassificationTask(name=COLA, loss_multiplier=1.0)],
 [Payload(COLA_train: tasks=[COLA], split=train),
  Payload(COLA_valid: tasks=[COLA], split=valid),
  Payload(COLA_test: tasks=[COLA], split=test)])

In [ ]:
from metal.mmtl.metal_model import MetalModel

model_baseline = MetalModel(tasks, seed=SEED, verbose=False)

In [ ]:
from metal.mmtl.trainer import MultitaskTrainer
trainer = MultitaskTrainer(seed=SEED)

In [ ]:
# trainer.train_model(
#     model_baseline,
#     payloads,
#     checkpoint_metric="COLA/COLA_valid/matthews_corr",
#     checkpoint_metric_mode="max",
#     writer="tensorboard",
#     optimizer="adamax",
#     lr=5e-5,
#     log_every=0.05,
#     score_every=0.1,
#     n_epochs=3,
#     progress_bar=True
# )

## Define slice over question marks

In [ ]:
%%time
from metal.mmtl.glue.glue_tasks import create_tasks_and_payloads

# define slices
slice_dict = {  # A map of the slices that apply to each task
   "COLA": ["ends_with_question_mark"]
}

# Create tasks and payloads
tasks_slice, payloads_slice = create_tasks_and_payloads(
    task_names,
#     max_datapoints=1000,
    dl_kwargs={"batch_size": 16},
    slice_dict=slice_dict
)

Using random seed: 128037
Loading COLA Dataset



Added label_set with 8550 labels for task COLA:ends_with_question_mark to payload COLA_train.
Added label_set with 1042 labels for task COLA:ends_with_question_mark to payload COLA_valid.
Added label_set with 1063 labels for task COLA:ends_with_question_mark to payload COLA_test.
CPU times: user 8.51 s, sys: 976 ms, total: 9.48 s
Wall time: 10.4 s


In [ ]:
payloads_slice[1]

Payload(COLA_valid: tasks=[COLA,COLA:ends_with_question_mark], split=valid)

In [ ]:
model_baseline.score(payloads_slice[1])

> /dfs/scratch0/vschen/metal-mmtl/metal/mmtl/metal_model.py(213)score()
-> Ys, Ys_probs, Ys_preds = self.predict_with_gold(
(Pdb) list
208  	        else:
209  	            task_names = payload.task_names
210  	            target_metrics = {task_name: None for task_name in task_names}
211  	
212  	        import pdb; pdb.set_trace()
213  ->	        Ys, Ys_probs, Ys_preds = self.predict_with_gold(
214  	            payload, task_names, return_preds=True, **kwargs
215  	        )
216  	
217  	        metrics_dict = {}
218  	        for task_name in task_names:
(Pdb) task_names
['COLA', 'COLA:ends_with_question_mark']
(Pdb) payload
Payload(COLA_valid: tasks=[COLA,COLA:ends_with_question_mark], split=valid)
(Pdb) model.task_names
*** NameError: name 'model' is not defined
(Pdb) model.task_map
*** NameError: name 'model' is not defined
(Pdb) self.task_map
{'COLA': ClassificationTask(name=COLA, loss_multiplier=1.0)}
(Pdb) task_names
['COLA', 'COLA:ends_with_question_mark']
(Pdb) self.task_ma

In [ ]:
tasks, payloads_slice

In [ ]:
import numpy as np
def count_num_labels(labels):
    return np.sum(np.array(labels) != 0)

In [ ]:
dataset = payloads_slice[0].data_loader.dataset
for labelset_name, labels in dataset.labels.items():
    print(labelset_name, count_num_labels(labels))

In [ ]:
# %%time
# # Pickle tasks and payloads
# import dill
# with open("FILENAME", 'wb') as f:
#     dill.dump((tasks, payloads), f)

In [ ]:
# %%time
# # Unpickle tasks and payloads
# import dill
# with open("FILENAME", 'rb') as f:
#     (tasks, payloads) = dill.load(f)

In [ ]:
from metal.mmtl.metal_model import MetalModel

model = MetalModel(tasks, seed=SEED, verbose=False)

In [ ]:
from metal.mmtl.trainer import MultitaskTrainer
trainer = MultitaskTrainer(seed=SEED)

In [ ]:
trainer.train_model(
    model,
    payloads,
    checkpoint_metric="COLA/COLA_valid/matthews_corr",
    checkpoint_metric_mode="max",
    writer="tensorboard",
    optimizer="adamax",
    lr=5e-5,
    log_every=0.05,
    score_every=0.1,
    n_epochs=8,
    progress_bar=True
)

In [ ]:
print("WE WIN!")